#### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLm application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:
* Using language models
* Using PromptTemplates and OutputParsers
* Using LangChain Expression Language (LCEL) to chain components together
* Debugging and tracing your application using LangSmith
* Deploying your application with LangServe

In [1]:
### Open AI API Key and Open Source models -- Llama3, Gemma2, mistral--Groq
import os 
from dotenv import load_dotenv
load_dotenv() 
import openai 
openai.api_key=os.getenv('OPENAI_API_KEY')

groq_api_key= os.getenv('GROQ_API_KEY')
groq_api_key

'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'

In [3]:
from langchain_groq import ChatGroq 
from langchain_openai import ChatOpenAI

model=ChatGroq(model='Gemma2-9b-It',groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x11ca56780>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x11ca57950>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage,SystemMessage 
messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]
result=model.invoke(messages)

In [6]:
result

AIMessage(content='Here are a few ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous?** (This is the most formal way to say it.)\n\n**Informal:**\n\n* **Salut, ça va?** (This is a casual way to say it.)\n* **Coucou, comment vas-tu?** (This is a very informal way to say it, usually used with friends and family.)\n\n\nLet me know if you have any other phrases you\'d like translated!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 114, 'prompt_tokens': 21, 'total_tokens': 135, 'completion_time': 0.207272727, 'prompt_time': 0.002143855, 'queue_time': 0.243273535, 'total_time': 0.209416582}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c7031738-cd47-4b4d-93bc-8f4a9cf62156-0', usage_metadata={'input_tokens': 21, 'output_tokens': 114, 'total_tokens': 135})

In [7]:
from langchain_core.output_parsers import StrOutputParser 
parser=StrOutputParser()
parser.invoke(result)

'Here are a few ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous?** (This is the most formal way to say it.)\n\n**Informal:**\n\n* **Salut, ça va?** (This is a casual way to say it.)\n* **Coucou, comment vas-tu?** (This is a very informal way to say it, usually used with friends and family.)\n\n\nLet me know if you have any other phrases you\'d like translated!\n'

In [8]:
### Using LCEL - chain the components 
chain=model|parser 
chain.invoke(messages)

'**Hello** - Bonjour\n\n**How are you?** - Comment allez-vous ? (formal) or Comment vas-tu ? (informal) \n\n\nSo the full translation would be:\n\n**Bonjour, comment allez-vous ?** (formal) or **Bonjour, comment vas-tu ?** (informal)\n'

In [11]:
### Prompt Templates 
from langchain_core.prompts import ChatPromptTemplate 
generice_template= "Translatte the following  into {language}:"
prompt=ChatPromptTemplate.from_messages(
    [
        ('system',generice_template),
        ('user','{text}')
    ]
)


In [13]:
result=prompt.invoke({'language':'French','text':'Hello'})

In [14]:
result.to_messages()

[SystemMessage(content='Translatte the following  into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [15]:
chain=prompt|model|parser
chain.invoke({'language':'French','text':'Hello'})

'Bonjour \n'